### txt 만들기
2. txt파일 만들기
- path마다 따라가면서 각 감정의 이미지를 불러올 수 있는 반복문
- 각 이미지에 해당하난 json파일에 있는 bbox의 좌표를 평균내서 가져오기
- 이미지를 resizing한 비율을 보고 origin_bbox의 좌표들 비율 조정하기
- 조정된 bbox의 좌표들 0과 1사이로 정규화 하기
txt파일에 들어갈 클래스 지정하기  [0,1,2,3,4,5,6]                                                                         or  { '분노': 0, '중립': 1, '기쁨': 2, '당황': 3, - '불안': 4, '상처': 5, '슬픔': 6 }
(center_x , center_y, width,  height)로 들어갈 수 있게 확인


In [ ]:
import os
import pandas as pd
# IMG_ROOT = '/home/KDT-admin/work/data3/train/'
IMG_ROOT = '/home/KDT-admin/work/data6_last_1000_300/train/'
DATA_ROOT = '/home/KDT-admin/work/data/'
# JSON_ROOT = '/home/KDT-admin/work/data/anger/labeled/validation/'

what = 'train' # You must change together IMG_ROOT

idx_to_class = {
  0: 'anger',
  1: 'normal',
  2: 'happy',
  3: 'embarrass',
  4: 'anxiety',
  5: 'pain',
  6: 'sad'
              }

classes = ('분노',  '중립',  '기쁨',  '당황',  '불안',  '상처',  '슬픔')

In [ ]:
import json 
import matplotlib.pyplot as plt
import cv2
def resizeing_bbox(original_bbox, resized_width, resized_height, original_width, original_height):
  resized_bbox = [
    int(original_bbox[0] * (resized_width / original_width)),    # xmin
    int(original_bbox[1] * (resized_height / original_height)),  # ymin
    int(original_bbox[2] * (resized_width / original_width)),    # xmax
    int(original_bbox[3] * (resized_height / original_height))   # ymax
  ]
  return resized_bbox
  
def normalized_cord(resized_bbox, desired_width):
  '''
  normalized_x_center = (x_min + x_max) / (2 * img_width)
  normalized_y_center = (y_min + y_max) / (2 * img_height)
  normalized_width = (x_max - x_min) / img_width
  normalized_height = (y_max - y_min) / img_height
  '''
  normalized_bbox = [
    (resized_bbox[0] + resized_bbox[2])/(2*desired_width),
    (resized_bbox[1] + resized_bbox[3])/(2*desired_width),
    (resized_bbox[2] - resized_bbox[0])/(desired_width),
    (resized_bbox[3] - resized_bbox[1])/(desired_width),
  ]
  return normalized_bbox

def annot_average(json_root,image_path):
  # /home/KDT-admin/work/data/anger/labeled/train/
  json_path = json_root + image_path + '.json'
    
  with open(json_path) as f:
    js = json.loads(f.read())
    # df = pd.json_normalize(js)
    maxX = (js['annot_A']['boxes']['maxX']+js['annot_B']['boxes']['maxX']+js['annot_C']['boxes']['maxX'])/3
    maxY = (js['annot_A']['boxes']['maxY']+js['annot_B']['boxes']['maxY']+js['annot_C']['boxes']['maxY'])/3
    minX = (js['annot_A']['boxes']['minX']+js['annot_B']['boxes']['minX']+js['annot_C']['boxes']['minX'])/3
    minY = (js['annot_A']['boxes']['minY']+js['annot_B']['boxes']['minY']+js['annot_C']['boxes']['minY'])/3
  
  # [xmin, ymin, xmax, ymax]
  return [minX,minY,maxX,maxY]

def load_image(path):

    img = cv2.imread(path)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    return img

# Plot numpy array
def plot_image(img):
    plt.imshow(img)
    plt.title(img.shape)

In [ ]:
import os
import torch
import torchvision.transforms as transforms
from PIL import Image, ImageDraw, ImageOps

# 원하는 크기 설정
desired_width = 640
desired_height = 640
cnt = 0
for num in idx_to_class.keys():
  
  directory = os.path.join(IMG_ROOT, idx_to_class[num])
  image_files = [os.path.splitext(os.path.basename(f))[0] for f in os.listdir(directory) if f.endswith(".jpg")]

# 이미지를 로드합니다. 예시로 'image.jpg' 파일을 사용합니다.
  
  for name in image_files:
    cnt += 1
    print(cnt)
    image_path = os.path.join(IMG_ROOT,idx_to_class[num], name + '.jpg')
    image = Image.open(image_path)
    image = ImageOps.exif_transpose(image)
    # 바운딩 박스 좌표 JSON bbox 평균
    JSON_ROOT = DATA_ROOT + idx_to_class[num] + '/labeled/' + what + '/'

    original_bbox = annot_average(JSON_ROOT, name) # [xmin, ymin, xmax, ymax]

    # 원본 이미지의 크기를 저장합니다.
    original_width, original_height = image.size

    # torchvision.transforms.Compose를 사용하여 resize 변환을 정의합니다.
    transform = transforms.Compose([
        transforms.Resize((desired_height, desired_width)),  # 원하는 크기로 resize
    ])

    # 이미지를 resize합니다.
    resized_image = transform(image)

    # 이미지의 크기를 다시 확인합니다.
    resized_width, resized_height = resized_image.size

    # 이미지의 크기가 변경되었으므로 바운딩 박스의 좌표도 변경해야 합니다.
    # 바운딩 박스의 좌표를 원본 이미지와 resizing된 이미지의 비율에 맞게 조정합니다.
    resized_bbox = resizeing_bbox(original_bbox, resized_width, resized_height, original_width, original_height)

    # resizing에 맞게 변경된 바운딩 박스의 좌표를 Yolo에 넣기 위해서
    # normalization과 각 좌표에 맞게 넣어줍니다.
    normalized_bbox = [num] + normalized_cord(resized_bbox,desired_width)

    # 이미지 위에 바운딩 박스를 그립니다.
    # draw = ImageDraw.Draw(resized_image)
    # draw.rectangle(resized_bbox, outline="red", width=3)  # 바운딩 박스를 빨간색으로 그립니다.

    with open(f"./labels/{name}.txt", 'w') as file:
      for item in normalized_bbox:
        file.write(str(item)+' ')

    # 변환된 이미지와 바운딩 박스를 사용하여 원본 이미지와 동일한 비율을 유지하면서 resizing된 이미지를 확인합니다.
    resized_image.save(f"./images/{name}.jpg")
    # resized_image.show()
